In [1]:
import codecs
import networkx as nx
import pandas as pd
import pickle
import re
from gensim.models import KeyedVectors
from os import listdir
from os.path import isfile, join
from utils import printProgressBar

# column number
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21
fichier_origine = 43

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\user\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Extract all valid cstr (Xeur, Xette, reX, ...)

In [2]:
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
valid_cstr = set()
counter = 0
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                elements = line.replace(' ', '').split('\t')
                valid_cstr.add(elements[cstr_1].replace('1', '').replace('2', ''))
                valid_cstr.add(elements[cstr_2].replace('1', '').replace('2', ''))
    counter += 1
    printProgressBar(counter, len(input_files), prefix='Progress:', suffix='complete', length=50, decimals=2)

Progress: |██████████████████████████████████████████████████| 100.00% complete


In [3]:
valid_cstr.update(['rX', 'réX'])
print(sorted(list(valid_cstr)))

['X', 'Xable', 'Xade', 'Xage', 'Xagier', 'Xaie', 'Xaille', 'Xailler', 'Xaillette', 'Xainat', 'Xaire', 'Xairie', 'Xais', 'Xaison', 'Xal', 'Xalat', 'Xalette', 'Xaliser', 'Xanat', 'Xance', 'Xande', 'Xange', 'Xant', 'Xariat', 'Xariser', 'Xassier', 'Xat', 'Xataire', 'Xate', 'Xaticat', 'Xation', 'Xatique', 'Xaume', 'Xel', 'Xelat', 'Xelet', 'Xelette', 'Xellette', 'Xence', 'Xenet', 'Xent', 'Xer', 'Xeraie', 'Xeret', 'Xerette', 'Xerie', 'Xerète', 'Xet', 'Xetier', 'Xetière', 'Xette', 'Xettes', 'Xeté', 'Xeur', 'Xeuse', 'Xeute', 'Xeutique', 'Xeux', 'Xfère', 'Xgenèse', 'Xgénique', 'Xiat', 'Xiau', 'Xicat', 'Xice', 'Xichette', 'Xiculet', 'Xie', 'Xien', 'Xier', 'Xiette', 'Xif', 'Xifier', 'Xikat', 'Xilier', 'Xillette', 'Xinat', 'Xine', 'Xinet', 'Xinette', 'Xinettes', 'Xing', 'Xinière', 'Xion', 'Xionat', 'Xionnaire', 'Xique', 'Xiquet', 'Xiquette', 'Xis', 'Xise', 'Xiser', 'Xisme', 'Xissariat', 'Xiste', 'Xistique', 'Xite', 'Xitique', 'Xité', 'Xium', 'Xière', 'Xlet', 'Xlette', 'Xlogie', 'Xlogique', 'Xment',

# Extract all lexemes in each family

In [4]:
family_dict = dict()
with codecs.open('summary_of_families.txt', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        if line_num == 0:
            continue
        cols = line.split('\t')
        lexemes = cols[2].replace('{','').replace('}','').replace('\'','').replace('\n','').split(', ')
        family_dict[cols[0]] = set(lexemes)
print(len(family_dict))

13178


# String comparison

In [12]:
# section to obtain families that contain two given families
L = nx.DiGraph()
vertex_attribute_dict = dict()
with codecs.open(join('posets', 'families_simplified_maxgraph.dot'), 'r', encoding='utf-8') as f:
    for line in f:
        if '->' in line:  # a line showing edges between concepts
            elements = line.split()
            L.add_edge(vertex_attribute_dict[elements[2]], vertex_attribute_dict[elements[0]])
        elif 'shape' in line:  # a line describing a concept 
            attribute_number = re.search('Attribute (.*)\|', line.replace('\\n', '')).group(1)
            vertex_attribute_dict[line.split()[0]] = attribute_number
            L.add_node(attribute_number)

def common_superfamily(family_number_1, family_number_2):
    group_1 = str(int(family_number_1.split('-')[0].replace('F', '')))
    group_2 = str(int(family_number_2.split('-')[0].replace('F', '')))
    children1 = nx.descendants(L, group_1)
    children2 = nx.descendants(L, group_2)
    common_children = children1.intersection(children2)
    if len(common_children) == 0:
        return 'no common superfamily'
    return_string = ''
    for child in common_children:
        return_string += 'F' + child.rjust(5, '0') + ' '
    return return_string[:-1]

In [15]:
# with valid cstr
family_dict_keys = list(family_dict.keys())
df_valid = pd.DataFrame(columns=['family_id_1', 'family_id_2', 'graph_1', 'cat_1', 'cstr_1', 'graph_2', 'cat_2', 'cstr_2', 'superfamilies'])
for k1 in range(0, len(family_dict)):
    #if family_dict_keys[k1] != 'F01545':
        #continue
    for k2 in range(k1+1, len(family_dict)):
        set1 = family_dict[family_dict_keys[k1]]
        set2 = family_dict[family_dict_keys[k2]]
        #if family_dict_keys[k2] != 'F03722-5':
            #continue
        connected = False
        for s1 in set1:
            [g1, c1] = s1.split('_')
            for s2 in set2:
                [g2, c2] = s2.split('_')
                if g1 in g2 and g2.replace(g1, 'X') in valid_cstr:
                    family_id_1 = family_dict_keys[k1]
                    family_id_2 = family_dict_keys[k2]
                    cstr = g2.replace(g1, 'X')
                    graph1 = g1
                    graph2 = g2
                    cat1 = c1
                    cat2 = c2
                    connected = True
                    break
                elif g2 in g1 and g1.replace(g2, 'X') in valid_cstr:
                    family_id_1 = family_dict_keys[k2]
                    family_id_2 = family_dict_keys[k1]
                    cstr = g1.replace(g2, 'X')
                    graph1 = g2
                    graph2 = g1
                    cat1 = c2
                    cat2 = c1
                    connected = True
                    break
            if connected:
                break
        if connected:
            df_valid = df_valid.append(pd.Series({
                'family_id_1': family_id_1,
                'family_id_2': family_id_2,
                'graph_1': graph1,
                'graph_2': graph2,
                'cat_1': cat1,
                'cat_2': cat2,
                'cstr_1': 'X',
                'cstr_2': cstr,
                'superfamilies': common_superfamily(family_dict_keys[k1], family_dict_keys[k2])
            }), ignore_index=True)
    printProgressBar(k1 + 1, len(family_dict), prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
df_valid.to_excel('unconnected,_valid_cstr.xls', index=False)

Progress: |██████████████████████████████████████████████████| 100.00% complete


C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


PermissionError: [Errno 13] Permission denied: 'unconnected,_valid_cstr.xls'

In [16]:
df_valid.to_excel('unconnected,_valid_cstr.xls', index=False)

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


In [ ]:
# valid + invalid cstr
family_dict_keys = list(family_dict.keys())
min_ratio = 0.5
df_valid_invalid = pd.DataFrame(columns=['family_id_1', 'family_id_2', 'graph_1', 'cat_1', 'cstr_1',\
                                         'graph_2', 'cat_2', 'cstr_2', 'ratio'])
for k1 in range(0, len(family_dict)):
    for k2 in range(k1+1, len(family_dict)):
        set1 = family_dict[family_dict_keys[k1]]
        set2 = family_dict[family_dict_keys[k2]]
        connected = False
        for s1 in set1:
            [g1, c1] = s1.split('_')
            for s2 in set2:
                [g2, c2] = s2.split('_')
                if g1 in g2 and len(g1) / len(g2) > min_ratio:
                    family_id_1 = family_dict_keys[k1]
                    family_id_2 = family_dict_keys[k2]
                    cstr = g2.replace(g1, 'X')
                    graph1 = g1
                    graph2 = g2
                    cat1 = c1
                    cat2 = c2
                    ratio = len(g1) / len(g2)
                    connected = True
                    break
                elif g2 in g1 and len(g2) / len(g1) > min_ratio:
                    family_id_1 = family_dict_keys[k2]
                    family_id_2 = family_dict_keys[k1]
                    cstr = g1.replace(g2, 'X')
                    graph1 = g2
                    graph2 = g1
                    cat1 = c2
                    cat2 = c1
                    ratio = len(g2) / len(g1)
                    connected = True
                    break
            if connected:
                break
        if connected:
            df_valid_invalid = df_valid_invalid.append(pd.Series({
                'family_id_1': family_id_1,
                'family_id_2': family_id_2,
                'graph_1': graph1,
                'graph_2': graph2,
                'cat_1': cat1,
                'cat_2': cat2,
                'cstr_1': 'X',
                'cstr_2': cstr,
                'ratio': ratio
            }), ignore_index=True)
    printProgressBar(k1 + 1, len(family_dict), prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
df_valid_invalid.to_excel('unconnected.xls', index=False)

In [6]:
# invalid cstr
df_valid_invalid = pd.read_excel('unconnected.xls')
df_valid = pd.read_excel('unconnected,_valid_cstr.xls')
df_valid_invalid['family_id_join'] = df_valid_invalid.apply(lambda row: sorted([row['family_id_1'], row['family_id_2']])[0]\
                                                            + '+' + sorted([row['family_id_1'], row['family_id_2']])[1],\
                                                            axis=1)
df_valid['family_id_join'] = df_valid.apply(lambda row: sorted([row['family_id_1'], row['family_id_2']])[0] + '+' + \
                                            sorted([row['family_id_1'], row['family_id_2']])[1], axis=1)
df_valid = df_valid[['family_id_join']]
df_invalid = df_valid.merge(df_valid_invalid, how = 'outer', on = ['family_id_join'],\
                                              indicator=True).loc[lambda x : x['_merge']=='right_only']
df_invalid.drop(['family_id_join', '_merge'], axis=1, inplace=True)
df_invalid.to_excel('unconnected,_invalid_cstr.xls', index=False)

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


# word2vec

In [ ]:
model = KeyedVectors.load_word2vec_format('frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin', binary=True, unicode_errors="ignore")

In [ ]:
model.most_similar('approchement')

In [ ]:
model.similarity('boucher', 'déboucher')